<a href="https://colab.research.google.com/github/waishun78/hungry-rag/blob/main/notebooks/answer_verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load model with Float16 or Int8




In [1]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install SentencePiece
!pip install evaluate
!pip install bert_score

In [2]:
!nvidia-smi

Tue Nov 21 03:01:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Larger LLM Inferences

In [3]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer


load_in_8bit = True
model_name_l = "lmsys/vicuna-7b-v1.5"

model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [4]:
def response_gen(input_ids):
  temperature=0.7
  with torch.no_grad():
      generation_output = model_l.generate(
          input_ids=input_ids,
          temperature=temperature,
          top_p = 1.0,
          do_sample=True,
          return_dict_in_generate=True,
          max_new_tokens=200,
      )
  s = generation_output.sequences[0][len(input_ids[0]):]
  output = tokenizer_l.decode(s)
  return output

### Preparing Example Context

In [5]:
examples = {
  "businesses": [
    {
      "id": "ahlZC7XOKN-XJ7P4sPZrRg",
      "alias": "cé-la-vi-restaurant-marina",
      "name": "CÉ LA VI Restaurant",
      "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/MwKP6-B9B_vfQzaeljQ0nA/o.jpg",
      "is_closed": False,
      "url": "https://www.yelp.com/biz/c%C3%A9-la-vi-restaurant-marina?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "review_count": 147,
      "categories": [
        {
          "alias": "bars",
          "title": "Bars"
        },
        {
          "alias": "asianfusion",
          "title": "Asian Fusion"
        }
      ],
      "rating": 3,
      "coordinates": {
        "latitude": 1.28337384753509,
        "longitude": 103.860615968168
      },
      "transactions": [],
      "price": "$$$",
      "location": {
        "address1": "1 Bayfront Avenue",
        "address2": "Tower 3",
        "address3": "",
        "city": "Marina",
        "zip_code": "018971",
        "country": "SG",
        "state": "SG",
        "display_address": [
          "1 Bayfront Avenue",
          "Tower 3",
          "Marina 018971",
          "Singapore"
        ]
      },
      "phone": "+6565082188",
      "display_phone": "+65 6508 2188",
      "distance": 163.2049653404058
    },
    {
      "id": "G0F3tLI-oQjQHxPYDmjy9A",
      "alias": "sushiairways-sushibar-singapore",
      "name": "SushiAirways Sushibar",
      "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/AJHTHCVI9QqbquXf89EbPw/o.jpg",
      "is_closed": False,
      "url": "https://www.yelp.com/biz/sushiairways-sushibar-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "review_count": 8,
      "categories": [
        {
          "alias": "japanese",
          "title": "Japanese"
        },
        {
          "alias": "sushi",
          "title": "Sushi Bars"
        }
      ],
      "rating": 3.5,
      "coordinates": {
        "latitude": 1.30117,
        "longitude": 103.8597336
      },
      "transactions": [],
      "price": "$$$",
      "location": {
        "address1": "20A Baghdad Street",
        "address2": "",
        "address3": "",
        "city": "Singapore",
        "zip_code": "199659",
        "country": "SG",
        "state": "SG",
        "display_address": [
          "20A Baghdad Street",
          "Singapore 199659",
          "Singapore"
        ]
      },
      "phone": "+6562911151",
      "display_phone": "+65 6291 1151",
      "distance": 1914.4506122870896
    },
    {
      "id": "-5_EPQpzNOKVgC3mxnQOWw",
      "alias": "kazokutei-singapore",
      "name": "Kazokutei",
      "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/sU8lUu7HYS3Ni6RbaduVLQ/o.jpg",
      "is_closed": False,
      "url": "https://www.yelp.com/biz/kazokutei-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "review_count": 5,
      "categories": [
        {
          "alias": "japanese",
          "title": "Japanese"
        }
      ],
      "rating": 5,
      "coordinates": {
        "latitude": 1.29966,
        "longitude": 103.8557129
      },
      "transactions": [],
      "location": {
        "address1": "200 Victoria Street, #02-49 Bugis Junction",
        "address2": "",
        "address3": "",
        "city": "Singapore",
        "zip_code": "188021",
        "country": "SG",
        "state": "SG",
        "display_address": [
          "200 Victoria Street, #02-49 Bugis Junction",
          "Singapore 188021",
          "Singapore"
        ]
      },
      "phone": "",
      "display_phone": "",
      "distance": 1016.3640048519915
    }
  ],
  "total": 3,
  "region": {
    "center": {
      "longitude": 103.8592529296875,
      "latitude": 1.283919115051798
    }
  }
}

In [6]:
example_context = [{"id":query.get("id",None), "alias":query.get("alias",None),"categories": query.get("categories", None), "name":query.get("name",None), "rating":query.get("rating",None), "price":query.get("price",None)} for query in examples["businesses"]]

In [7]:
eg_review_1 = {
  "reviews": [
    {
      "id": "l3tV3U24cTK7-uC-7t79IQ",
      "url": "https://www.yelp.com/biz/c%C3%A9-la-vi-restaurant-marina?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=l3tV3U24cTK7-uC-7t79IQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Vibe and view are amazing we made reservation and were sat by the edge overseeing garden by the bay. Service by Mannan and Jenne were amazing and were very...",
      "rating": 5,
      "time_created": "2023-10-05 04:40:07",
      "user": {
        "id": "OIZ-9EjFQWlqiga4Bl-GGQ",
        "profile_url": "https://www.yelp.com/user_details?userid=OIZ-9EjFQWlqiga4Bl-GGQ",
        "image_url": "https://s3-media4.fl.yelpcdn.com/photo/yV4m83M_QTj915bXHdouyQ/o.jpg",
        "name": "Yunjing H."
      }
    },
    {
      "id": "cNHGGFvgLQtbVkgHzTY_8w",
      "url": "https://www.yelp.com/biz/c%C3%A9-la-vi-restaurant-marina?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=cNHGGFvgLQtbVkgHzTY_8w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "So we didn't dine at Ce La Vi but ordered a beer at the bar adjacent to the Marina Bay Sands Observation Deck. The beer was $13 SGD which was way more than...",
      "rating": 3,
      "time_created": "2023-10-03 07:58:05",
      "user": {
        "id": "_pBzBgtCTN9PNUPfgPDI8A",
        "profile_url": "https://www.yelp.com/user_details?userid=_pBzBgtCTN9PNUPfgPDI8A",
        "image_url": "https://s3-media2.fl.yelpcdn.com/photo/hf5_r3w-4RULlMbVpdiH8w/o.jpg",
        "name": "Trina D."
      }
    },
    {
      "id": "tQrYIHO_rhOIdkrsHgC61w",
      "url": "https://www.yelp.com/biz/c%C3%A9-la-vi-restaurant-marina?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=tQrYIHO_rhOIdkrsHgC61w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "If you are in Singapore then you have to visit the marina sands hotel. While at the sands you need to see the spectacular views of the city from the top. Ce...",
      "rating": 3,
      "time_created": "2023-08-09 21:13:07",
      "user": {
        "id": "J7U2JFwC1OXWm6GxIRLa3g",
        "profile_url": "https://www.yelp.com/user_details?userid=J7U2JFwC1OXWm6GxIRLa3g",
        "image_url": "https://s3-media3.fl.yelpcdn.com/photo/WdQZAt1AqMxmZIyMjGgCWw/o.jpg",
        "name": "Alla L."
      }
    }
  ],
  "total": 147,
  "possible_languages": [
    "de",
    "en",
    "ja",
    "nb",
    "fr"
  ]
}

eg_review_1 = [{"text":query.get("text",None), "rating":query.get("rating",None), "url":query.get("url",None)} for query in eg_review_1["reviews"]]

In [8]:
eg_review_2 = {
  "reviews": [
    {
      "id": "b9xwFaynCNqtadYUT383Ug",
      "url": "https://www.yelp.com/biz/sushiairways-sushibar-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=b9xwFaynCNqtadYUT383Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Stopped in for a light late dinner. I liked the whole decor and flight theme. They really went into detail. The menu was a flight manual, lights were like...",
      "rating": 5,
      "time_created": "2017-01-19 14:24:50",
      "user": {
        "id": "gekp2Z5I4vPGf2NeFnZX2A",
        "profile_url": "https://www.yelp.com/user_details?userid=gekp2Z5I4vPGf2NeFnZX2A",
        "image_url": "https://s3-media3.fl.yelpcdn.com/photo/4yVP0O8lbgFuLFeGYW9SNw/o.jpg",
        "name": "Maria-Elena T."
      }
    },
    {
      "id": "KxEW22LD8_4cXnrnUR9V1A",
      "url": "https://www.yelp.com/biz/sushiairways-sushibar-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=KxEW22LD8_4cXnrnUR9V1A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Food was good, but way too pricey. Worth it if you go during lunch for their lunch special offer... But I visited in the evening and the offer wasn't...",
      "rating": 3,
      "time_created": "2016-06-17 04:38:38",
      "user": {
        "id": "kTtpHWjpzvbhGxAxAUC02Q",
        "profile_url": "https://www.yelp.com/user_details?userid=kTtpHWjpzvbhGxAxAUC02Q",
        "image_url": "https://s3-media3.fl.yelpcdn.com/photo/b3-PJITinHK8KmdC0Yli5A/o.jpg",
        "name": "Wan Yi S."
      }
    },
    {
      "id": "CqR4lM0jkEdcUtFCCtVvhw",
      "url": "https://www.yelp.com/biz/sushiairways-sushibar-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=CqR4lM0jkEdcUtFCCtVvhw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Happened on Sushi Airways by sheer luck as I was in town for a couple of weeks and holed up across the street at the Pan Pacific Suites.  I figured it would...",
      "rating": 5,
      "time_created": "2015-03-10 06:32:48",
      "user": {
        "id": "x0qmWq7M3CGx3lBebouGOw",
        "profile_url": "https://www.yelp.com/user_details?userid=x0qmWq7M3CGx3lBebouGOw",
        "image_url": "https://s3-media2.fl.yelpcdn.com/photo/0bmTuYlhdP7I8QGLgNl5Uw/o.jpg",
        "name": "Michael W."
      }
    }
  ],
  "total": 8,
  "possible_languages": [
    "en"
  ]
}

eg_review_2 = [{"text":query.get("text",None), "rating":query.get("rating",None), "url":query.get("url",None)} for query in eg_review_2["reviews"]]

In [9]:
eg_review_3 = {
  "reviews": [
    {
      "id": "XfrB0vJWyPiE2p9O_9yNgg",
      "url": "https://www.yelp.com/biz/kazokutei-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=XfrB0vJWyPiE2p9O_9yNgg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "I don't even know where to start with this one. Really romantic and delicious with good view. Enjoyed the vibey atmosphere",
      "rating": 5,
      "time_created": "2016-02-21 22:35:05",
      "user": {
        "id": "IzwsiilxLRG_jg3wKIUxpg",
        "profile_url": "https://www.yelp.com/user_details?userid=IzwsiilxLRG_jg3wKIUxpg",
        "image_url": "https://s3-media4.fl.yelpcdn.com/photo/n3wo5WpwL5x4MRJJ-s7opw/o.jpg",
        "name": "Marcus W."
      }
    },
    {
      "id": "Xaog6-cyQgkR5BNkyLvNDg",
      "url": "https://www.yelp.com/biz/kazokutei-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=Xaog6-cyQgkR5BNkyLvNDg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Kazokutei serves conventional Japanese fare - sushi, soba, donburi - at budget-friendly prices (meaning under $15 for set meals). We decided to give this a...",
      "rating": 5,
      "time_created": "2012-08-13 03:49:11",
      "user": {
        "id": "hC1HjO-Tye6dar3Bg0aZMw",
        "profile_url": "https://www.yelp.com/user_details?userid=hC1HjO-Tye6dar3Bg0aZMw",
        "image_url": "https://s3-media3.fl.yelpcdn.com/photo/pvQ1NZZ0ALsrMNkY0go8sw/o.jpg",
        "name": "Huiyin S."
      }
    },
    {
      "id": "p211Y1PTRhWDMLbCyjr7sA",
      "url": "https://www.yelp.com/biz/kazokutei-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=p211Y1PTRhWDMLbCyjr7sA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "We were just looking for a place to grab lunch and decided on cruising through Bugis Junction.\n\nWe selected Kazokutei.\n\nThe staff seated us quickly.  Soon...",
      "rating": 5,
      "time_created": "2012-09-29 03:42:29",
      "user": {
        "id": "3S53VaMDPy8Lywk2TuPb4A",
        "profile_url": "https://www.yelp.com/user_details?userid=3S53VaMDPy8Lywk2TuPb4A",
        "image_url": "https://s3-media1.fl.yelpcdn.com/photo/RnHth4N0ioWRWVJMIZKccQ/o.jpg",
        "name": "Mark D."
      }
    }
  ],
  "total": 5,
  "possible_languages": [
    "en"
  ]
}

eg_review_3 = [{"text":query.get("text",None), "rating":query.get("rating",None), "url":query.get("url",None)} for query in eg_review_3["reviews"]]

In [10]:
for i, v in zip(example_context, [eg_review_1, eg_review_2, eg_review_3]):
  i["reviews"] = v
example_context

[{'id': 'ahlZC7XOKN-XJ7P4sPZrRg',
  'alias': 'cé-la-vi-restaurant-marina',
  'categories': [{'alias': 'bars', 'title': 'Bars'},
   {'alias': 'asianfusion', 'title': 'Asian Fusion'}],
  'name': 'CÉ LA VI Restaurant',
  'rating': 3,
  'price': '$$$',
  'reviews': [{'text': 'Vibe and view are amazing we made reservation and were sat by the edge overseeing garden by the bay. Service by Mannan and Jenne were amazing and were very...',
    'rating': 5,
    'url': 'https://www.yelp.com/biz/c%C3%A9-la-vi-restaurant-marina?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=l3tV3U24cTK7-uC-7t79IQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg'},
   {'text': "So we didn't dine at Ce La Vi but ordered a beer at the bar adjacent to the Marina Bay Sands Observation Deck. The beer was $13 SGD which was way more than...",
    'rating': 3,
    'url': 'https://www.yelp.com/biz/c%C3%A9-la-vi-restaurant-marina?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=cNHGGFvgLQtb

#### Preparing context input

In [11]:
yelp_context = {
  "businesses": [
    {
      "id": "WjpaNwaJB9IX7gja2BhBzA",
      "alias": "tandoori-flame-boon-tat-singapore",
      "name": "Tandoori Flame @ Boon Tat",
      "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/wAg83o3_Hgl0ehoG2EvgGA/o.jpg",
      "is_closed": False,
      "url": "https://www.yelp.com/biz/tandoori-flame-boon-tat-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "review_count": 9,
      "categories": [
        {
          "alias": "indpak",
          "title": "Indian"
        }
      ],
      "rating": 5,
      "coordinates": {
        "latitude": 1.28116774503181,
        "longitude": 103.848429529976
      },
      "transactions": [],
      "price": "$$",
      "location": {
        "address1": "5 Boon Tat St",
        "address2": "#01-01",
        "address3": "",
        "city": "Singapore",
        "zip_code": "069613",
        "country": "SG",
        "state": "SG",
        "display_address": [
          "5 Boon Tat St",
          "#01-01",
          "Singapore 069613",
          "Singapore"
        ]
      },
      "phone": "+6562233200",
      "display_phone": "+65 6223 3200",
      "distance": 1259.5633889750538
    },
    {
      "id": "NzviQebksMfqVQbAESXAJg",
      "alias": "annalakshmi-singapore-2",
      "name": "Annalakshmi",
      "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/7MAaFixkO0X1WkzcavLM4Q/o.jpg",
      "is_closed": False,
      "url": "https://www.yelp.com/biz/annalakshmi-singapore-2?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "review_count": 23,
      "categories": [
        {
          "alias": "indpak",
          "title": "Indian"
        },
        {
          "alias": "vegetarian",
          "title": "Vegetarian"
        }
      ],
      "rating": 4,
      "coordinates": {
        "latitude": 1.28777,
        "longitude": 103.8435211
      },
      "transactions": [],
      "price": "$",
      "location": {
        "address1": "Blk 20, Havelock Road",
        "address2": "#01-04, Central Square",
        "address3": "",
        "city": "Singapore",
        "zip_code": "059765",
        "country": "SG",
        "state": "SG",
        "display_address": [
          "Blk 20, Havelock Road",
          "#01-04, Central Square",
          "Singapore 059765",
          "Singapore"
        ]
      },
      "phone": "+6563399993",
      "display_phone": "+65 6339 9993",
      "distance": 1404.212400847113
    },
    {
      "id": "8JOF2_8ILNy_QmHb4Bywqg",
      "alias": "gajalee-singapore",
      "name": "Gajalee",
      "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/TjcurpSkGlpFvbz1ksJRgg/o.jpg",
      "is_closed": False,
      "url": "https://www.yelp.com/biz/gajalee-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "review_count": 8,
      "categories": [
        {
          "alias": "indpak",
          "title": "Indian"
        },
        {
          "alias": "seafood",
          "title": "Seafood"
        }
      ],
      "rating": 4.5,
      "coordinates": {
        "latitude": 1.289591,
        "longitude": 103.855825
      },
      "transactions": [],
      "price": "$$",
      "location": {
        "address1": "8 Raffles Ave",
        "address2": "#02-12",
        "address3": "",
        "city": "Singapore",
        "zip_code": "039802",
        "country": "SG",
        "state": "SG",
        "display_address": [
          "8 Raffles Ave",
          "#02-12",
          "Singapore 039802",
          "Singapore"
        ]
      },
      "phone": "+6567333278",
      "display_phone": "+65 6733 3278",
      "distance": 18.148491064581055
    }
  ],
  "total": 131,
  "region": {
    "center": {
      "longitude": 103.85581970214844,
      "latitude": 1.2897541275200428
    }
  }
}

context = [{"id":query.get("id",None), "alias":query.get("alias",None),"categories": query.get("categories", None), "name":query.get("name",None), "rating":query.get("rating",None), "price":query.get("price",None)} for query in yelp_context["businesses"]]
context = context[:3]
context

[{'id': 'WjpaNwaJB9IX7gja2BhBzA',
  'alias': 'tandoori-flame-boon-tat-singapore',
  'categories': [{'alias': 'indpak', 'title': 'Indian'}],
  'name': 'Tandoori Flame @ Boon Tat',
  'rating': 5,
  'price': '$$'},
 {'id': 'NzviQebksMfqVQbAESXAJg',
  'alias': 'annalakshmi-singapore-2',
  'categories': [{'alias': 'indpak', 'title': 'Indian'},
   {'alias': 'vegetarian', 'title': 'Vegetarian'}],
  'name': 'Annalakshmi',
  'rating': 4,
  'price': '$'},
 {'id': '8JOF2_8ILNy_QmHb4Bywqg',
  'alias': 'gajalee-singapore',
  'categories': [{'alias': 'indpak', 'title': 'Indian'},
   {'alias': 'seafood', 'title': 'Seafood'}],
  'name': 'Gajalee',
  'rating': 4.5,
  'price': '$$'}]

In [12]:
context_review_4 = {
  "reviews": [
    {
      "id": "ta8xkTsYs4CsKc4q--MGhw",
      "url": "https://www.yelp.com/biz/tandoori-flame-boon-tat-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=ta8xkTsYs4CsKc4q--MGhw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Pro tip: \n- Recommend the Chicken tiki Masla. Skip the Chicken Tandoori.\n\nWhile I'll admit I'm no expert in Indian food, I still feel this places had some...",
      "rating": 4,
      "time_created": "2020-01-23 05:23:51",
      "user": {
        "id": "mqxwq4sqdI8lf1iHD2aQRw",
        "profile_url": "https://www.yelp.com/user_details?userid=mqxwq4sqdI8lf1iHD2aQRw",
        "image_url": "https://s3-media4.fl.yelpcdn.com/photo/Fe1Fiza1sIt3zk9NsIUCHA/o.jpg",
        "name": "George P."
      }
    },
    {
      "id": "h-GcwoC5hDd5DY6JnDjz9Q",
      "url": "https://www.yelp.com/biz/tandoori-flame-boon-tat-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=h-GcwoC5hDd5DY6JnDjz9Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Great indian food in the CBD, located near Raffles Place/Telok Ayer. Prices are reasonable and restaurant is very clean. Highly recommend the chicken tikka...",
      "rating": 5,
      "time_created": "2018-03-19 18:59:41",
      "user": {
        "id": "0nSzAzzqVUY1W-D-IVvu_A",
        "profile_url": "https://www.yelp.com/user_details?userid=0nSzAzzqVUY1W-D-IVvu_A",
        "image_url": "https://s3-media3.fl.yelpcdn.com/photo/v5ZKMHGy8eS8GurgmIwCLw/o.jpg",
        "name": "Nayoung W."
      }
    },
    {
      "id": "rW6MAX_u0o62WKKiojI6ww",
      "url": "https://www.yelp.com/biz/tandoori-flame-boon-tat-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=rW6MAX_u0o62WKKiojI6ww&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "A small menu, food that is not chilli'ed to oblivion, balanced and skillful cooking are complemented by very friendly and efficient service.\n\nWe were the...",
      "rating": 5,
      "time_created": "2018-10-18 06:32:28",
      "user": {
        "id": "jvj9YfBp_XPM-3z1Es_P8w",
        "profile_url": "https://www.yelp.com/user_details?userid=jvj9YfBp_XPM-3z1Es_P8w",
        "image_url": "https://s3-media3.fl.yelpcdn.com/photo/6Mr4aSvWXyWsCP6hDC3Ymw/o.jpg",
        "name": "Mouette G."
      }
    }
  ],
  "total": 9,
  "possible_languages": [
    "en"
  ]
}

context_review_4 = [{"text":query.get("text",None), "rating":query.get("rating",None), "url":query.get("url",None)} for query in context_review_4["reviews"]]

In [13]:
context_review_5 = {
  "reviews": [
    {
      "id": "eat3GTBZRRkvjPY32It0Xg",
      "url": "https://www.yelp.com/biz/annalakshmi-singapore-2?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=eat3GTBZRRkvjPY32It0Xg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Annalakshmi is part of a temple and serves Indian buffet style cuisine. There is no set price for the meal because this place takes donation only. I think...",
      "rating": 4,
      "time_created": "2018-12-24 21:35:47",
      "user": {
        "id": "joFAJDSxQpFd_uBbUi-grA",
        "profile_url": "https://www.yelp.com/user_details?userid=joFAJDSxQpFd_uBbUi-grA",
        "image_url": "https://s3-media2.fl.yelpcdn.com/photo/_r0D3YlijKoU2h3WBkIVPw/o.jpg",
        "name": "Michelle T."
      }
    },
    {
      "id": "_GRHjZgb1yQSGlN3hb0XFg",
      "url": "https://www.yelp.com/biz/annalakshmi-singapore-2?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=_GRHjZgb1yQSGlN3hb0XFg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Excellent food which captures the essence of South Indian cuisine. Chanced upon this place when searching for a place to eat. Interestingly they don't have...",
      "rating": 5,
      "time_created": "2019-09-06 00:09:34",
      "user": {
        "id": "v9ecHXhf7LXNZiaN4MlB7A",
        "profile_url": "https://www.yelp.com/user_details?userid=v9ecHXhf7LXNZiaN4MlB7A",
        "image_url": "https://s3-media1.fl.yelpcdn.com/photo/lvCo_4CvXngb8JYL4pUafA/o.jpg",
        "name": "Radhika K."
      }
    },
    {
      "id": "3paOy15NDsleWeRDUdhPgA",
      "url": "https://www.yelp.com/biz/annalakshmi-singapore-2?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=3paOy15NDsleWeRDUdhPgA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "We were visiting Singapore and while searching for a good Indian restaurant we found this place. It was a fantastic experience and food \n\nWhen is the last...",
      "rating": 5,
      "time_created": "2018-09-08 00:26:57",
      "user": {
        "id": "qgqqese0s7ImXvE0qKVBWw",
        "profile_url": "https://www.yelp.com/user_details?userid=qgqqese0s7ImXvE0qKVBWw",
        "image_url": None,
        "name": "Sudarshan H."
      }
    }
  ],
  "total": 23,
  "possible_languages": [
    "en",
    "de"
  ]
}

context_review_5 = [{"text":query.get("text",None), "rating":query.get("rating",None), "url":query.get("url",None)} for query in context_review_5["reviews"]]

In [14]:
context_review_6 = {
  "reviews": [
    {
      "id": "wSwAwFPralsjIHUy3q0jJA",
      "url": "https://www.yelp.com/biz/gajalee-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=wSwAwFPralsjIHUy3q0jJA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Amazing Indian Seafood restaurant \n\nGreat value!  \n\nNice wine selection. \n\nGreen chili pomfret",
      "rating": 5,
      "time_created": "2017-09-20 05:05:13",
      "user": {
        "id": "fc0E3w6TYCU-qguDeCEHsQ",
        "profile_url": "https://www.yelp.com/user_details?userid=fc0E3w6TYCU-qguDeCEHsQ",
        "image_url": "https://s3-media2.fl.yelpcdn.com/photo/y8g97qi0QQcqXkfTUHAcnQ/o.jpg",
        "name": "Agnes T."
      }
    },
    {
      "id": "MKwT3U46nG2Q7qxsBZIlRg",
      "url": "https://www.yelp.com/biz/gajalee-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=MKwT3U46nG2Q7qxsBZIlRg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "Not your typical Indian seafood restaurant..\n\nUnique dishes with good excellent service.. Try the bombili and solkadi for starters.. Very authentic...",
      "rating": 4,
      "time_created": "2015-10-15 08:01:10",
      "user": {
        "id": "1tQfj737uhgdKcQ-5akTdA",
        "profile_url": "https://www.yelp.com/user_details?userid=1tQfj737uhgdKcQ-5akTdA",
        "image_url": "https://s3-media1.fl.yelpcdn.com/photo/snrPD9k0S3TLbBvo6_w23g/o.jpg",
        "name": "Ashish C."
      }
    },
    {
      "id": "xUoUmd9Y8bTggtSdvnNI7g",
      "url": "https://www.yelp.com/biz/gajalee-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=xUoUmd9Y8bTggtSdvnNI7g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg",
      "text": "We accidentally came in here because another place we wanted to go to is closed for Friday lunch (i know, what?). Originally i was hesitant since, while i...",
      "rating": 4,
      "time_created": "2014-12-16 20:10:54",
      "user": {
        "id": "WjFI4zVmeDkA7c3tcOy3-g",
        "profile_url": "https://www.yelp.com/user_details?userid=WjFI4zVmeDkA7c3tcOy3-g",
        "image_url": "https://s3-media3.fl.yelpcdn.com/photo/XONSOKTUVpAosI3kCDsmLA/o.jpg",
        "name": "Melissa H."
      }
    }
  ],
  "total": 8,
  "possible_languages": [
    "en"
  ]
}

context_review_6 = [{"text":query.get("text",None), "rating":query.get("rating",None), "url":query.get("url",None)} for query in context_review_6["reviews"]]

In [15]:
for i, v in zip(context, [context_review_4[:1], context_review_5[:1], context_review_6[:1]]):
  i["reviews"] = v

context

[{'id': 'WjpaNwaJB9IX7gja2BhBzA',
  'alias': 'tandoori-flame-boon-tat-singapore',
  'categories': [{'alias': 'indpak', 'title': 'Indian'}],
  'name': 'Tandoori Flame @ Boon Tat',
  'rating': 5,
  'price': '$$',
  'reviews': [{'text': "Pro tip: \n- Recommend the Chicken tiki Masla. Skip the Chicken Tandoori.\n\nWhile I'll admit I'm no expert in Indian food, I still feel this places had some...",
    'rating': 4,
    'url': 'https://www.yelp.com/biz/tandoori-flame-boon-tat-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=ta8xkTsYs4CsKc4q--MGhw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg'}]},
 {'id': 'NzviQebksMfqVQbAESXAJg',
  'alias': 'annalakshmi-singapore-2',
  'categories': [{'alias': 'indpak', 'title': 'Indian'},
   {'alias': 'vegetarian', 'title': 'Vegetarian'}],
  'name': 'Annalakshmi',
  'rating': 4,
  'price': '$',
  'reviews': [{'text': 'Annalakshmi is part of a temple and serves Indian buffet style cuisine. There is no set

In [16]:
input = {
    "question": "Where to get Indian food in Singapore that is near the Esplanade and suitable for a group of 10 people?",
    "clarifier":"""1. How spicy do you prefer your Indian food to be?
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. How many people are in your group?
                  5. What time are you planning to have your meal?</s>
                  """,
    "additional_user_criteria":"""
                  1. NIL
                  2. NIL
                  3. Cheap
                  4. NIL
                  """,
    "requirements": "Indian [Cuisine], $$ [Price], group of 10 people [General-Atmosphere], near Esplanade [Location], 19:00 [Opening]",
    "context": context
}

#### Run

In [17]:
import json

eg = '\n\n'.join([json.dumps(restaurant) for restaurant in example_context])
supplement_context = '\n\n'.join([json.dumps(restaurant) for restaurant in input['context']])


prompt2 = f"""
  Can you answer the query using the additional user input as criteria and choose the best three restaurant options from the context and explain your choice with quotes from the context? An example is shown, please output the answer for the ACTUAL case
  EXAMPLE OF OUTPUT:
  Query:
  Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
  Additional User Input:
  1. Sushi
  2. Mostly just sushi
  3. $$
  4. Intimate with nice lighting, Inside Marina Bay Sands
  5. 22:00 minimally
  Context:
  {eg}
  Answer:
    According to my research,
    1. Kazokutei is the best option with a good rating of '5'. It is 'Really romantic and delicious with good view' and 'serves conventional Japanese fare - sushi, soba, donburi'
    2. CÉ LA VI Restaurant is a good alternative, it 'Vibe and view are amazing', and also has 'asian fusion food'. It is also more of a fine-dining at '$$$'.

  """ + f"""
  ACTUAL:
  Query:
  {input['question']}
  Additional User Input:
  {input['additional_user_criteria']}
  Context:
  {supplement_context}
  Answer:
  """

print(prompt2)


  Can you answer the query using the additional user input as criteria and choose the best three restaurant options from the context and explain your choice with quotes from the context? An example is shown, please output the answer for the ACTUAL case
  EXAMPLE OF OUTPUT:
  Query:
  Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
  Additional User Input:
  1. Sushi
  2. Mostly just sushi
  3. $$
  4. Intimate with nice lighting, Inside Marina Bay Sands
  5. 22:00 minimally
  Context:
  {"id": "ahlZC7XOKN-XJ7P4sPZrRg", "alias": "c\u00e9-la-vi-restaurant-marina", "categories": [{"alias": "bars", "title": "Bars"}, {"alias": "asianfusion", "title": "Asian Fusion"}], "name": "C\u00c9 LA VI Restaurant", "rating": 3, "price": "$$$", "reviews": [{"text": "Vibe and view are amazing we made reservation and were sat by the edge overseeing garden by the bay. Service by Mannan and Jenne were amazing and were very...", "rating": 5, "url": 

In [18]:
j = tokenizer_l(prompt2, return_tensors='pt')
input_ids = j["input_ids"].to("cuda")

output = response_gen(input_ids)
print("Prediction: \n", output)

del input_ids
torch.cuda.empty_cache()

Prediction: 
 
According to my research,
1. Tandoori Flame @ Boon Tat is the closest Indian restaurant to the Esplanade with a good rating of '5'. It also serves Indian food and is suitable for a group of 10 people.
2. Gajalee is a good alternative as it is also an Indian seafood restaurant and has a good rating of '4.5'. However, it is not as close to the Esplanade as Tandoori Flame @ Boon Tat.
3. Annalakshmi is a vegetarian Indian restaurant that is part of a temple and serves food on a donation basis. It is not as close to the Esplanade as the other two options, but it is a unique dining experience and has a good rating of '4'.</s>


# BERTScore

In [ ]:
import evaluate

bertscore = evaluate.load("bertscore")

In [25]:
# Related restaurant
for i in range(len(input['context'])):
  bert_score = bertscore.compute(predictions=[output], references=[str(input['context'][i])], lang="en")
  print(f"\n\nRelated BERTScore {i}: {bert_score}\n", str(input['context'][i]), output)

# Unrelated restaurant
for i in range(len(example_context)):
  bert_score = bertscore.compute(predictions=[output], references=[str(example_context[i])], lang="en")
  print(f"\n\nUnrelated BERTScore {i}: {bert_score}\n", str(example_context[i]), output)




Related BERTScore 0: {'precision': [0.8213313817977905], 'recall': [0.7410725951194763], 'f1': [0.7791406512260437], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.2)'}
 {'id': 'WjpaNwaJB9IX7gja2BhBzA', 'alias': 'tandoori-flame-boon-tat-singapore', 'categories': [{'alias': 'indpak', 'title': 'Indian'}], 'name': 'Tandoori Flame @ Boon Tat', 'rating': 5, 'price': '$$', 'reviews': [{'text': "Pro tip: \n- Recommend the Chicken tiki Masla. Skip the Chicken Tandoori.\n\nWhile I'll admit I'm no expert in Indian food, I still feel this places had some...", 'rating': 4, 'url': 'https://www.yelp.com/biz/tandoori-flame-boon-tat-singapore?adjust_creative=KiNVvQ0Keh4NSvR570v8Cg&hrid=ta8xkTsYs4CsKc4q--MGhw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KiNVvQ0Keh4NSvR570v8Cg'}]} 
According to my research,
1. Tandoori Flame @ Boon Tat is the closest Indian restaurant to the Esplanade with a good rating of '5'. It also serves Indian food and is suitable f